# Jim's Machine Learning Testbed



## Jupyter Notes
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

# Basic Python interpreter check

In [ ]:
# Does anything work
print("Hello World!")

# An IPython (Jupyter) thing
%env



Use a core Python package, numpy:

In [ ]:
import numpy as np

np.add(1, 2)

# A basic `TensorFlow` installation check

A sanity check call to `tensorflow` to make sure it finds and loads the library. NOTE: this does not do any tensor processing:

In [ ]:
import tensorflow as tf

print(f"A TF computation: {tf.add(1, 5).numpy()}")

hello = tf.constant('Hello, TensorFlow!')
print(f"A TensorFlow string constant: {hello.numpy()}")

# Of special `import`
The following odd import is to get around an IDE type checking issue which is a problem either in `tensorflow` package of an IDE shortcoming.

In [ ]:
from typing import TYPE_CHECKING

# These imports only affect an IDE like PyCharm
from keras.src.layers.core.dense import Dense
from keras.src.models import Model
from keras.src.models.sequential import Sequential
from keras.src.layers.regularization.dropout import Dropout


Now we load a dataset from the convenient tensorflow_datasets python package.

In [ ]:
# Load from tensorflow_dataset
# Import directly because it's in the same directory
from jims_ml.tensorflow_datasets import greet

# Use the function
print(greet("DataSpell User"))


# Playing with my Pipes

In [17]:
from keras.api import optimizers, losses, metrics, activations
from keras import Input
from jims_ml.sandbox.tf_tinker import ModelTest, RunBatch

# define a batch include a common dataset for all tests
run_batch = RunBatch()
run_batch.x_train = tf.constant([[1.0], [2.0], [3.0], [4.0]])
run_batch.y_train = tf.constant([[2.0], [4.0], [6.0], [8.0]])
run_batch.x_test = tf.constant([[5.0], [6.0]])
run_batch.y_test = tf.constant([[10.0], [12.0]])

# define one or more models
model = Sequential([
    Input(shape=(1,)),  # Input layer for single-dimensional input
    Dense(32, activations.relu),  # kera.src.core.dense.Dense
    # Dense(16, activation='sigmoid'),
    # Dense(16, activation='softmax'),
    
    Dense(1, activations.linear)
])

test_common_dense = ModelTest().set_description("Common Dense Sequence").set_model(model)
test_common_dense.set_fit_epochs(10)

# invoke model compile here
test_common_dense.compile_model(optimizer=optimizers.Adam(), loss=losses.MeanSquaredError(),
                                metrics=[metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError])

# stochastic gradient decent cross-entropy loss
# test_common_dense.compile_model(optimizer='sgd', loss='cel', metrics=['mae']) 

# test_common_dense.model.fit()

run_batch.add_test(test_common_dense)

# run all models against the same batch dataset
run_batch.run_all()

print('\nTest accuracy:', run_batch.results[0].eval_results)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Testing Model: Common Dense Sequence
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_25 (Dense)                │ (None, 32)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step - loss: 23.0185 - mean_absolute_error: 4.3797 - mean_absolute_percentage_error: 87.5948
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 22.7680 - mean_absolute_error: 4.3550 - mean_absolute_percentage_error: 87.0720
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 22.5187 - mean_absolute_error: 4.3303 - mean_absolute_percentage_error: 86.5487
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 22.2704 - mean_absolute_error: 4.3056 - mean_absolute_percentage_error: 86.0249
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 22.0234 - mean_absolute_error: 4.2808 - mean_absolute_percentage_error: 85.5006
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 21.7776 - mean_absolute_error: 4.2560 - mean_absolute_percentage_error: 84.9759
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 21.5330 - mean_absolute_error: 4.2312 - mean_absolute_percentage_error: 84.4508
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2

ValueError: too many values to unpack (expected 2)